
# DLASeR framework

import the code

In [1]:
from src import DLASeR
from src import Operator

Using TensorFlow backend.


The goals

**T1**: PL < 10%  
**T2**: LA < 5%  
**O1**: minimize EC

In [2]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.optimizers import Adam, RMSprop
input_size = 85 # The dimensionality of the input features
class_layers = [30,15,5]
class_scaler = StandardScaler()
reg_scaler = MinMaxScaler()
reg_layers = [30,50,15,5]
batch_size = 128
class_optimizer = Adam
reg_optimizer = RMSprop
class_lr = 3e-3
reg_lr = 5e-3

In [3]:
dlaser = DLASeR(['packetloss', 'latency', 'energyconsumption'], input_size)
dlaser.add_threshold_goal('packetloss', Operator.Min, 10, class_scaler, class_layers) # Add threshold goal: PL < 10%
dlaser.add_threshold_goal('latency', Operator.Min, 5, class_scaler, class_layers) # Add threshold goal: LA < 5%
dlaser.add_optimization_goal('energyconsumption', Operator.Min, reg_scaler, reg_layers) # Add setpoint goal: EC == 12.9 (with epsilon of 0.1)
dlaser.print_goals()
print('\n')
print('Model names:', dlaser.get_model_names())

---------- THRESHOLD GOALS ----------
packetloss < 10
latency < 5
--------- OPTIMIZATION GOALS ---------
Minimize energyconsumption


Model names: ['packetloss_thresh10', 'latency_thresh5', 'energyconsumption_opt']


In [4]:
dlaser.compile(optimizers={'packetloss_thresh10':class_optimizer, 'latency_thresh5':class_optimizer, 'energyconsumption_opt':reg_optimizer}, 
               lrs={'packetloss_thresh10':class_lr, 'latency_thresh5':class_lr, 'energyconsumption_opt':reg_lr})
dlaser.print_models()


~~~~~~~~~~~~~~~~~~~~~~~~~~packetloss_thresh10
Scaler: StandardScaler
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 85)]              0         
_________________________________________________________________
dense (Dense)                (None, 30)                2580      
_________________________________________________________________
dropout (Dropout)            (None, 30)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 15)                465       
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 80        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6         
Total params: 3,131
Trainable params: 3,131
Non-trainable


# DLASeR+ framework

import the code

In [1]:
from src import DLASeRPlus
from src import Operator

Using TensorFlow backend.


The goals

**T1**: LA < 5%  
**O1**: minimize PL  
**S1**: EC == 12.9

The hyper parameters

In [2]:
from keras.optimizers import RMSprop
from sklearn.preprocessing import StandardScaler
input_size = 85 # The dimensionality of the input features
core_layers = [100,50,25]
scaler = StandardScaler
class_layers = [30,15,5]
reg_layers = [30,50,15,5]
batch_size = 128
optimizer = RMSprop
lr = 3e-3

In [3]:
dlaserp = DLASeRPlus(['packetloss', 'latency', 'energyconsumption'], input_size, scaler, core_layers)
dlaserp.add_threshold_goal('latency', Operator.Min, 5, class_layers) # Add threshold goal: LA < 5%
dlaserp.add_optimization_goal('packetloss', Operator.Min, reg_layers) # Add optimization goal: minimize packet loss
dlaserp.add_setpoint_goal('energyconsumption', 12.9, 0.1, class_layers) # Add setpoint goal: EC == 12.9 (with epsilon of 0.1)
dlaserp.compile(optimizer=optimizer, lr=lr)
dlaserp.print_goals()
print('\n')
dlaserp.print_model()

---------- THRESHOLD GOALS ----------
latency < 5
---------- SET-POINT GOALS ----------
energyconsumption == 12.9
--------- OPTIMIZATION GOALS ---------
Minimize packetloss


Scaler: type
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         [(None, 85)]         0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 100)          8600        main_input[0][0]                 
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 100)          400         dense[0][0]                      
__________________________________________________________________________________________________
den